In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale
import os

import boto3
from io import BytesIO
import sagemaker.amazon.common as smac
from sagemaker import get_execution_role

In [9]:
'''
-----------------------------------------------------------------------------
--- Inptuts
-----------------------------------------------------------------------------
'''
dataL = 4
s3DataSetName = 'L6currentData.npz'
DataName = 'dataScaledL{}'.format(dataL)
interpDataName = 'interpDataScaledL{}.csv'.format(dataL)
polarDataName = 'polarDataScaledL{}.csv'.format(dataL)
polarInterpDataName = 'polarInterpDataScaledL{}.csv'.format(dataL)

In [10]:
'''
-----------------------------------------------------------------------------
--- Definitions
-----------------------------------------------------------------------------
'''
# Configuring S3
s3_bucket_name = 'jasper-ml-sagemaker'
role = get_execution_role()

client = boto3.client('s3')
resource = boto3.resource('s3')
my_bucket = resource.Bucket(s3_bucket_name)

def interpolateData(depth, current, interval):
    
    n = round((depth[-1] - depth[0] + interval) / interval)
    x = np.linspace(depth[0], depth[-1], n)
    yinterp = np.zeros((len(current), int(n)))
    
    for i in range(len(current)):
        yinterp[i] = np.interp(x, depth, current[i])
    
    return yinterp


def cartToPolar(current1, current2):
    """
    Input:
    Two orthogonal current profiles.
    Output:
    Single df; alternating magnitude & direction rows
    """
    polarData = np.zeros((len(current1), 2*len(current1[0])))
    
    for i in range(len(current1[0])):
        polarData[:,2*i] = (current1[:,i]**2 + current2[:,i]**2)**0.5
        polarData[:,2*i+1] = np.arctan2(current1[:,i], current2[:,i])
        
    return polarData

In [20]:
'''
-----------------------------------------------------------------------------
--- Importing Data
-----------------------------------------------------------------------------
'''
dataLocation = 'Data/'
dataKey = s3DataSetName
obj = client.get_object(Bucket=s3_bucket_name, Key=dataLocation + dataKey)
dataSet = np.load(BytesIO(obj['Body'].read()))

EWcur = dataSet['U']
NScur = dataSet['V']
t = dataSet['t']
depth = dataSet['depth']

In [27]:
'''
-----------------------------------------------------------------------------
--- Scaling & Interpolation
-----------------------------------------------------------------------------
'''
# Scaling

originalData = np.append(EWcur,NScur, axis=1)

scaledData = scale(originalData)

dataKey = DataName
np.savetxt(dataKey, scaledData, delimiter=',')
#my_bucket.upload_file(dataKey, Key=dataKey)
#os.remove(dataKey)


# Interpolating & Scaling
EWinterp = interpolateData(depth,EWcur,10)
NSinterp = interpolateData(depth,NScur,10)

interpData = np.append(EWinterp,NSinterp, axis=1)

scaledInterpData = scale(interpData)

dataKey = interpDataName
np.savetxt(dataKey, scaledInterpData, delimiter=',')
#my_bucket.upload_file(dataKey, Key=dataKey)
os.remove(dataKey)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:17: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.


In [53]:
'''
-----------------------------------------------------------------------------
--- Polar & Polar Interp
-----------------------------------------------------------------------------
'''
# Polar Data
polarData = cartToPolar(EWcur,NScur)

scaledPolarData = scale(polarData)

dataKey = polarDataName
np.savetxt(dataKey, scaledPolarData, delimiter=',')
my_bucket.upload_file(dataKey, Key=dataKey)
os.remove(dataKey)

# Polar Interped Data
EWinterp = interpolateData(depth,EWcur,10)
NSinterp = interpolateData(depth,NScur,10)

polarInterpData = cartToPolar(EWinterp,NSinterp)

scaledPolarInterpData = scale(polarData)

dataKey = polarInterpDataName
np.savetxt(dataKey, scaledPolarInterpData, delimiter=',')
my_bucket.upload_file(dataKey, Key=dataKey)
os.remove(dataKey)



/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:17: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
